In [16]:
import pandas as pd

In [17]:
# wallets_table.csv
# transactions_table.csv

In [18]:
# combined_trx = pd.read_csv('final_combined_wallets_with_trx_balances.csv')

In [19]:
# combined_trx.head(5)

In [20]:
# # TAKE OUT 3rd row as Raw TRX (UNCOMMENT TO CREATE JSON OF ANY ROW)
# import json

# wallet_row = combined_trx.iloc[2]

# # Extract the address and transactions
# wallet_address = wallet_row['Address']
# transactions_str = wallet_row['Transactions']

# # Convert the string to a JSON object (if it's stored as a string of list of dicts)
# transactions_json = json.loads(transactions_str.replace("'", '"'))  # convert to valid JSON

# # Create a JSON structure
# wallet_data = {
#     "Address": wallet_address,
#     "Transactions": transactions_json
# }

# # Save to a JSON file
# with open(f'{wallet_address}_transactions.json', 'w') as f:
#     json.dump(wallet_data, f, indent=2)

# print(f"Saved {wallet_address}_transactions.json")


In [21]:
# # Set testing mode
# testing = True

# if testing:
#     combined_trx = combined_trx.head(20)

In [22]:
# import ast

# def extract_transaction_features(tx_list):
#     if not tx_list:
#         return {
#             'total_txn': 0,
#             'total_gas_used': 0,
#             'unique_to_addresses': 0,
#             'unique_from_addresses': 0,
#             'value_sent': 0,
#             'value_received': 0,
#             'internal_txns': 0,
#             'failed_txns': 0,
#             'high_gas_txns': 0,
#         }

#     from_addresses = set()
#     to_addresses = set()
#     total_gas = 0
#     value_sent = 0
#     value_received = 0
#     failed_txns = 0
#     internal_txns = 0
#     high_gas_txns = 0

#     for tx in tx_list:
#         from_addr = tx.get('from', '').lower()
#         to_addr = tx.get('to', '').lower()
#         value = int(tx.get('value', 0))
#         gas = int(tx.get('gasUsed', 0))
#         is_error = tx.get('isError', '0') == '1'
#         method_id = tx.get('methodId', '')
#         input_data = tx.get('input', '')
        
#         # Build sets
#         from_addresses.add(from_addr)
#         to_addresses.add(to_addr)

#         # Aggregate
#         total_gas += gas
#         if is_error:
#             failed_txns += 1
#         if gas > 500000:  # Custom threshold
#             high_gas_txns += 1
#         if value > 0:
#             if method_id == '0x' or input_data == '0x':
#                 value_sent += value
#             else:
#                 value_received += value
#         if tx.get('contractAddress'):
#             internal_txns += 1

#     return {
#         'total_txn': len(tx_list),
#         'total_gas_used': total_gas,
#         'unique_to_addresses': len(to_addresses),
#         'unique_from_addresses': len(from_addresses),
#         'value_sent': value_sent,
#         'value_received': value_received,
#         'internal_txns': internal_txns,
#         'failed_txns': failed_txns,
#         'high_gas_txns': high_gas_txns,
#     }


In [23]:
# import ast

# # Convert 'Transactions' column from string to actual list of dicts
# combined_trx['Transactions'] = combined_trx['Transactions'].apply(
#     lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) and x != "[]" else []
# )


# # Apply feature extraction to each wallet
# feature_df = combined_trx['Transactions'].apply(extract_transaction_features).apply(pd.Series)

# # Combine with original dataframe
# df_features = pd.concat([combined_trx, feature_df], axis=1)


In [24]:
# df_features.to_csv('wallet_features_extracted.csv', index=False)
# print("✅ Feature-enhanced CSV saved as 'wallet_features_extracted.csv'")

In [25]:
# df_features.head(10)

TO CHECK IF THE ADDRESS IS TOKEN OR NOT

In [26]:
# import requests
# import time

# def is_contract(address, api_key):
#     url = "https://api.etherscan.io/api"
#     params = {
#         "module": "contract",
#         "action": "getsourcecode",
#         "address": address,
#         "apikey": api_key
#     }
#     response = requests.get(url, params=params)
#     result = response.json().get('result', [{}])[0]
#     return result.get("ABI") != "Contract source code not verified"


In [27]:
# testing = False
# api_key = "7A32HIJRFBSUJQK8S6WD3E2XCDS6XBW9JZ"

# if testing:
#     test_addresses = combined_trx['Address'].head(20)
# else:
#     test_addresses = combined_trx['Address']

# non_contract_addresses = []

# for addr in test_addresses:
#     if not is_contract(addr, api_key):
#         non_contract_addresses.append(addr)
#     time.sleep(0.2)  # respect rate limits

# # Filter your DataFrame
# combined_trx = combined_trx[combined_trx['Address'].isin(non_contract_addresses)]

In [28]:
# non_contract_addresses

In [29]:
# output_file = 'combined_wallets_with_transactions_and_balances_1.csv'
# combined_trx.to_csv(output_file, index=False)

In [ ]:
combined_trx = pd.read_csv('../../data/combined_wallets_with_transactions_and_balances_1.csv')

### LABEL SET

In [ ]:
import json

with open("labels_sanctioned.json") as f:
    sanctioned = set(json.load(f))

with open("labels_mixers.json") as f:
    mixers = set(json.load(f))

LABEL_MAPS = {
    "sanctioned": sanctioned,
    "mixers": mixers,
    "bridges": set(),    # fill in later
    "cex": set(),
    "defi": set(),
    "scam_tokens": set()
}


### OLD CODE FOR DATA GENERATION

In [ ]:
# import pandas as pd
# import json
# from datetime import datetime
# import numpy as np
# from collections import defaultdict
# from scipy.stats import entropy

# def extract_transaction_features(transactions_json):
#     try:
#         transactions = json.loads(transactions_json.replace("'", "\"")) if isinstance(transactions_json, str) else transactions_json
#     except Exception:
#         return {key: 0 for key in [
#             "total_transactions", "self_transfer_ratio", "circular_txn_count", "circular_txn_ratio",
#             "avg_txn_value_eth", "txn_spike_score", "value_std_dev", "avg_gas_used", "avg_gas_price",
#             "active_days", "wallet_age_days", "unique_counterparties", "failed_txn_ratio",
#             "eth_inflow_outflow_ratio", "erc20_txn_count", "nft_txn_count", "first_txn_time_of_day",
#             "erc20_token_diversity", "tx_direction_ratio", "contract_interaction_ratio",
#             "value_entropy", "tx_burst_count", "average_txn_interval", "new_token_interaction_count",
#             "token_approval_count", "sbt_poap_event_count", "approved_token_list"
#         ]}

#     if not transactions:
#         return {key: 0 for key in [
#             "total_transactions", "self_transfer_ratio", "circular_txn_count", "circular_txn_ratio",
#             "avg_txn_value_eth", "txn_spike_score", "value_std_dev", "avg_gas_used", "avg_gas_price",
#             "active_days", "wallet_age_days", "unique_counterparties", "failed_txn_ratio",
#             "eth_inflow_outflow_ratio", "erc20_txn_count", "nft_txn_count", "first_txn_time_of_day",
#             "erc20_token_diversity", "tx_direction_ratio", "contract_interaction_ratio",
#             "value_entropy", "tx_burst_count", "average_txn_interval", "new_token_interaction_count",
#             "token_approval_count", "sbt_poap_event_count", "approved_token_list"
#         ]}

#     txn_values, gas_used_list, gas_price_list, time_stamps = [], [], [], []
#     from_addresses, to_addresses, counterparties = set(), set(), set()
#     approved_tokens = set()
#     erc20_token_contracts, new_token_contracts, circular_pairs = set(), set(), {}
#     inflow = outflow = failed_txns = self_transfers = circular_count = contract_interactions = erc20_count = nft_count = approval_count = sbt_count = 0
#     intervals = []
#     prev_time = None

#     for tx in transactions:
#         from_addr = tx.get('from', '').lower()
#         to_addr = tx.get('to', '').lower()
#         value = int(tx.get('value', 0))
#         gas = int(tx.get('gasUsed', 0))
#         gas_price = int(tx.get('gasPrice', 0))
#         timestamp = int(tx.get('timeStamp', 0))
#         is_error = tx.get('isError', "0") == "1"
#         input_data = tx.get('input', '')
#         contract_address = tx.get('contractAddress', '')
#         method_id = tx.get('methodId', '').lower()

#         txn_values.append(value / 1e18)
#         gas_used_list.append(gas)
#         gas_price_list.append(gas_price)
#         time_stamps.append(timestamp)
#         from_addresses.add(from_addr)
#         to_addresses.add(to_addr)
#         counterparties.update([from_addr, to_addr])

#         if from_addr == to_addr:
#             self_transfers += 1

#         if (to_addr, from_addr) in circular_pairs:
#             circular_count += 1
#         else:
#             circular_pairs[(from_addr, to_addr)] = True

#         inflow += value if to_addr else 0
#         outflow += value if from_addr else 0
#         failed_txns += int(is_error)

#         if input_data.startswith("0xa9059cbb"):
#             erc20_count += 1
#             erc20_token_contracts.add(to_addr)
#         elif input_data.startswith("0x23b872dd"):
#             nft_count += 1
#         if input_data.startswith("0x095ea7b3"):
#             approval_count += 1
#             approved_tokens.add(to_addr)
#         if input_data.startswith("0x40c10f19") or input_data.startswith("0xb88d4fde"):
#             sbt_count += 1
#         if contract_address:
#             contract_interactions += 1
#         if method_id:
#             new_token_contracts.add(to_addr)
#         if prev_time:
#             intervals.append(timestamp - prev_time)
#         prev_time = timestamp

#     total_txns = len(transactions)
#     wallet_age = (datetime.utcnow() - datetime.utcfromtimestamp(min(time_stamps))).days if time_stamps else 0
#     days_active = len(set(datetime.utcfromtimestamp(ts).date() for ts in time_stamps))
#     txn_spike_score = max(pd.Series(time_stamps).value_counts()) if time_stamps else 0
#     time_of_day = datetime.utcfromtimestamp(min(time_stamps)).hour if time_stamps else 0
#     avg_interval = np.mean(intervals) if intervals else 0
#     value_entropy = entropy(pd.Series(txn_values).value_counts(normalize=True))

#     return {
#         "total_transactions": total_txns,
#         "self_transfer_ratio": self_transfers / total_txns,
#         "circular_txn_count": circular_count,
#         "circular_txn_ratio": circular_count / total_txns,
#         "avg_txn_value_eth": np.mean(txn_values),
#         "txn_spike_score": txn_spike_score,
#         "value_std_dev": np.std(txn_values),
#         "avg_gas_used": np.mean(gas_used_list),
#         "avg_gas_price": np.mean(gas_price_list),
#         "active_days": days_active,
#         "wallet_age_days": wallet_age,
#         "unique_counterparties": len(counterparties),
#         "failed_txn_ratio": failed_txns / total_txns,
#         "eth_inflow_outflow_ratio": inflow / (outflow + 1),
#         "erc20_txn_count": erc20_count,
#         "nft_txn_count": nft_count,
#         "first_txn_time_of_day": time_of_day,
#         "erc20_token_diversity": len(erc20_token_contracts),
#         "tx_direction_ratio": len(to_addresses) / (len(from_addresses) + 1),
#         "contract_interaction_ratio": contract_interactions / total_txns,
#         "value_entropy": value_entropy,
#         "tx_burst_count": txn_spike_score,  # proxy for now
#         "average_txn_interval": avg_interval,
#         "new_token_interaction_count": len(new_token_contracts),
#         "token_approval_count": approval_count,
#         "sbt_poap_event_count": sbt_count,
#         "approved_token_list": list(approved_tokens)
#     }


In [ ]:
import json
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
from scipy.stats import entropy

# ---- helpers ----
def _safe_num(x, default=0):
    try:
        return float(x)
    except Exception:
        return default

def _to_eth(wei):
    try:
        return float(wei) / 1e18
    except Exception:
        return 0.0

def _gini(x):
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    if x.size == 0:
        return 0.0
    if np.all(x == 0):
        return 0.0
    x = np.sort(x)
    n = x.size
    cumx = np.cumsum(x)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

def _fano(counts):
    counts = np.asarray(counts, dtype=float)
    if counts.size == 0:
        return 0.0
    mu = counts.mean()
    if mu <= 1e-12:
        return 0.0
    return counts.var(ddof=0) / mu

def _burstiness_from_series(ts_unix, bucket='hour'):
    if len(ts_unix) == 0:
        return 0.0
    ts = pd.to_datetime(ts_unix, unit='s', utc=True)
    if bucket == 'hour':
        grp = ts.floor('H')
    elif bucket == 'day':
        grp = ts.floor('D')
    else:
        grp = ts
    counts = pd.Series(1, index=grp).groupby(level=0).size().values
    return _fano(counts)

def _weekend_ratio(ts_unix):
    if len(ts_unix) == 0:
        return 0.0
    dow = pd.to_datetime(ts_unix, unit='s', utc=True).dayofweek.values  # Mon=0..Sun=6
    return float(np.mean(np.isin(dow, [5, 6])))

def _hour_entropy(ts_unix):
    if len(ts_unix) == 0:
        return 0.0
    hours = pd.to_datetime(ts_unix, unit='s', utc=True).hour.values
    hist = np.bincount(hours, minlength=24).astype(float)
    p = hist / (hist.sum() if hist.sum() else 1.0)
    p = p[p > 0]
    return float(-(p * np.log(p)).sum())

def _mask_window(ts_unix, now_ts, days):
    if len(ts_unix) == 0:
        return np.zeros(0, dtype=bool)
    lower = now_ts - days * 24 * 3600
    return (ts_unix >= lower) & (ts_unix <= now_ts)

def _reciprocity_ratio(my_addr, froms, tos):
    # share of counterparties that are both senders and receivers relative to me
    cf = set([a for a, t in zip(froms, tos) if a != my_addr])
    ct = set([b for a, b in zip(froms, tos) if b != my_addr])
    both = cf.intersection(ct)
    denom = len(cf.union(ct))
    return float(len(both) / denom) if denom > 0 else 0.0

def _short_back_and_forth(my_addr, froms, tos, ts_unix, max_days=7):
    # count A<->me ping-pong within a time window (simple A->me then me->A or vice-versa)
    if len(ts_unix) == 0:
        return 0
    order = np.argsort(ts_unix)
    froms = np.asarray(froms, dtype=object)[order]
    tos   = np.asarray(tos,   dtype=object)[order]
    ts    = np.asarray(ts_unix, dtype=int)[order]
    count = 0
    max_delta = max_days * 24 * 3600
    for i in range(len(ts) - 1):
        a1, b1 = froms[i], tos[i]
        a2, b2 = froms[i+1], tos[i+1]
        if ((b1 == my_addr and a2 == my_addr and a1 == b2) or
            (a1 == my_addr and b2 == my_addr and b1 == a2)):
            if ts[i+1] - ts[i] <= max_delta:
                count += 1
    return int(count)

def _peel_score_outflows(values_eth, ts_unix, min_len=3, within_hours=24):
    # heuristic: detect >=min_len consecutive outflows with strictly decreasing amounts in a short period
    if len(values_eth) < min_len:
        return 0
    order = np.argsort(ts_unix)
    v = np.asarray(values_eth, dtype=float)[order]
    t = np.asarray(ts_unix, dtype=int)[order]
    # simple scan
    streak = 1
    score = 0
    for i in range(1, len(v)):
        if (t[i] - t[i-1]) <= within_hours * 3600 and v[i] < v[i-1]:
            streak += 1
        else:
            if streak >= min_len:
                score += 1
            streak = 1
    if streak >= min_len:
        score += 1
    return int(score)

def extract_transaction_features(
    transactions_json,
    *,
    my_address=None,          # optional: pass the wallet address (lowercased) for reciprocity/back-and-forth
    label_maps=None,          # optional dict of sets: {'sanctioned': set(), 'mixers': set(), 'bridges': set(), 'cex': set(), 'defi': set(), 'scam_tokens': set()}
    now_ts=None               # optional unix timestamp (int). If None, uses current UTC.
):
    """
    Extended extractor: preserves your original keys and adds new, high-signal features.
    - Computes 7/30/90d windowed metrics.
    - Behavioral: burstiness, hour/day patterns, Gini, reciprocity, ping-pong, peel score.
    - Direct exposure ratios against provided label maps (value-weighted).
    """
    # ---------- parse ----------
    try:
        txs = (json.loads(transactions_json.replace("'", "\""))
               if isinstance(transactions_json, str) else transactions_json)
    except Exception:
        txs = []

    if not txs:
        zero_base = {
            "total_transactions": 0, "self_transfer_ratio": 0.0, "circular_txn_count": 0, "circular_txn_ratio": 0.0,
            "avg_txn_value_eth": 0.0, "txn_spike_score": 0, "value_std_dev": 0.0, "avg_gas_used": 0.0, "avg_gas_price": 0.0,
            "active_days": 0, "wallet_age_days": 0, "unique_counterparties": 0, "failed_txn_ratio": 0.0,
            "eth_inflow_outflow_ratio": 0.0, "erc20_txn_count": 0, "nft_txn_count": 0, "first_txn_time_of_day": 0,
            "erc20_token_diversity": 0, "tx_direction_ratio": 0.0, "contract_interaction_ratio": 0.0,
            "value_entropy": 0.0, "tx_burst_count": 0, "average_txn_interval": 0.0, "new_token_interaction_count": 0,
            "token_approval_count": 0, "sbt_poap_event_count": 0, "approved_token_list": []
        }
        # new features all zeroed
        zero_new = {
            "weekend_txn_ratio": 0.0,
            "hour_entropy": 0.0,
            "value_gini": 0.0,
            "burstiness_hour": 0.0,
            "burstiness_day": 0.0,
            "reciprocity_ratio": 0.0,
            "back_and_forth_7d": 0,
            "peel_chain_score_24h": 0,
            # windowed txn/value counts
            "tx_count_7d": 0, "tx_count_30d": 0, "tx_count_90d": 0,
            "inflow_eth_7d": 0.0, "inflow_eth_30d": 0.0, "inflow_eth_90d": 0.0,
            "outflow_eth_7d": 0.0, "outflow_eth_30d": 0.0, "outflow_eth_90d": 0.0,
            # direct exposure (value share)
            "exposure_mixer_30d": 0.0, "exposure_mixer_90d": 0.0,
            "exposure_bridge_30d": 0.0, "exposure_bridge_90d": 0.0,
            "exposure_sanctioned_30d": 0.0, "exposure_sanctioned_90d": 0.0,
            "exposure_cex_30d": 0.0, "exposure_cex_90d": 0.0,
            "exposure_defi_30d": 0.0, "exposure_defi_90d": 0.0,
        }
        zero_base.update(zero_new)
        return zero_base

    # ---------- arrays ----------
    froms, tos, vals_eth, gas_used, gas_price, ts_unix = [], [], [], [], [], []
    is_err = []
    input_data = []
    method_id = []
    contract_address = []

    for tx in txs:
        froms.append((tx.get('from') or '').lower())
        tos.append((tx.get('to') or '').lower())
        vals_eth.append(_to_eth(tx.get('value', 0)))
        gas_used.append(_safe_num(tx.get('gasUsed', 0)))
        gas_price.append(_safe_num(tx.get('gasPrice', 0)))
        ts_unix.append(int(_safe_num(tx.get('timeStamp', 0))))
        is_err.append(1 if str(tx.get('isError', "0")) == "1" else 0)
        input_data.append(tx.get('input', '') or '')
        method_id.append((tx.get('methodId', '') or '').lower())
        contract_address.append(tx.get('contractAddress', '') or '')

    froms = np.array(froms, dtype=object)
    tos   = np.array(tos,   dtype=object)
    vals_eth = np.array(vals_eth, dtype=float)
    gas_used = np.array(gas_used, dtype=float)
    gas_price= np.array(gas_price, dtype=float)
    ts_unix  = np.array(ts_unix,  dtype=int)
    is_err   = np.array(is_err,   dtype=int)
    N = len(ts_unix)

    # ---------- basics you already had ----------
    total_txns = int(N)
    self_transfers = int(np.sum(froms == tos))
    circular_pairs = set()
    circular_count = 0
    for a, b in zip(froms, tos):
        if (b, a) in circular_pairs:
            circular_count += 1
        circular_pairs.add((a, b))

    avg_val = float(vals_eth.mean()) if N else 0.0
    val_std = float(vals_eth.std()) if N else 0.0
    gas_used_avg = float(gas_used.mean()) if N else 0.0
    gas_price_avg= float(gas_price.mean()) if N else 0.0
    days_active = int(pd.to_datetime(ts_unix, unit='s', utc=True).floor('D').nunique())
    first_hour  = int(pd.to_datetime(ts_unix.min(), unit='s', utc=True).hour) if N else 0
    avg_interval = float(np.diff(np.sort(ts_unix)).mean()) if N > 1 else 0.0
    counterparties = set(froms).union(set(tos))
    failed_ratio = float(is_err.mean()) if N else 0.0

    # inflow/outflow by our perspective (if my_address is given)
    inflow_wei = 0.0
    outflow_wei = 0.0
    if my_address:
        ma = my_address.lower()
        inflow_wei = float(np.sum(vals_eth[(tos == ma)])) * 1e18
        outflow_wei= float(np.sum(vals_eth[(froms == ma)])) * 1e18
    # fallback if not provided: treat any "to" as inflow proxy (rough)
    if not my_address:
        inflow_wei = float(np.sum(vals_eth[~np.isnan(vals_eth)])) * 1e18
        outflow_wei= float(np.sum(vals_eth[~np.isnan(vals_eth)])) * 1e18

    # txn spike score (max tx per day)
    spike = 0
    if N:
        by_day = pd.to_datetime(ts_unix, unit='s', utc=True).floor('D')
        spike = int(pd.Series(1, index=by_day).groupby(level=0).size().max())

    # ERC-20 / NFT / approvals / SBT heuristics
    erc20_sel = np.array([s.startswith("0xa9059cbb") for s in input_data], dtype=bool)
    nft_sel   = np.array([s.startswith("0x23b872dd") for s in input_data], dtype=bool)
    appr_sel  = np.array([s.startswith("0x095ea7b3") for s in input_data], dtype=bool)
    sbt_sel   = np.array([s.startswith("0x40c10f19") or s.startswith("0xb88d4fde") for s in input_data], dtype=bool)
    erc20_cnt = int(erc20_sel.sum())
    nft_cnt   = int(nft_sel.sum())
    appr_cnt  = int(appr_sel.sum())
    sbt_cnt   = int(sbt_sel.sum())
    erc20_contract_set = set(tos[erc20_sel])
    new_token_contracts = set(tos[[bool(m) for m in method_id]])
    contract_interactions = int(np.sum([1 for ca in contract_address if ca]))
    tx_dir_ratio = float(len(set(tos)) / (len(set(froms)) + 1))

    # entropy of value buckets
    if N:
        vals_nonzero = vals_eth[vals_eth > 0]
        if len(vals_nonzero) > 0:
            vc = pd.Series(vals_nonzero.round(8)).value_counts(normalize=True)
            val_entropy = float(entropy(vc.values))
        else:
            val_entropy = 0.0
    else:
        val_entropy = 0.0

    # wallet age
    now_ts = int(now_ts if now_ts is not None else datetime.now(timezone.utc).timestamp())
    wallet_age_days = int((now_ts - ts_unix.min()) / (24 * 3600)) if N else 0

    # ---------- NEW: temporal windows ----------
    m7  = _mask_window(ts_unix, now_ts, 7)
    m30 = _mask_window(ts_unix, now_ts, 30)
    m90 = _mask_window(ts_unix, now_ts, 90)

    # If my_address supplied, we can compute directional inflow/outflow per window
    if my_address:
        ma = my_address.lower()
        inflow_7  = float(vals_eth[m7 & (tos == ma)].sum())
        inflow_30 = float(vals_eth[m30 & (tos == ma)].sum())
        inflow_90 = float(vals_eth[m90 & (tos == ma)].sum())
        out_7  = float(vals_eth[m7 & (froms == ma)].sum())
        out_30 = float(vals_eth[m30 & (froms == ma)].sum())
        out_90 = float(vals_eth[m90 & (froms == ma)].sum())
    else:
        # fallback: treat totals as both (neutral)
        inflow_7 = inflow_30 = inflow_90 = float(vals_eth[m7].sum())
        out_7    = out_30    = out_90    = float(vals_eth[m7].sum())

    # ---------- NEW: behavior & shape ----------
    weekend_ratio = _weekend_ratio(ts_unix)
    hour_ent = _hour_entropy(ts_unix)
    gini_val = _gini(vals_eth[vals_eth > 0])
    burst_h = _burstiness_from_series(ts_unix, 'hour')
    burst_d = _burstiness_from_series(ts_unix, 'day')
    reciprocity = _reciprocity_ratio(my_address.lower() if my_address else '', froms, tos)
    back_forth_7d = _short_back_and_forth(my_address.lower() if my_address else '', froms, tos, ts_unix, max_days=7)
    # peel score requires outflows
    if my_address:
        out_mask = (froms == my_address.lower())
        peel_score = _peel_score_outflows(vals_eth[out_mask], ts_unix[out_mask], min_len=3, within_hours=24)
    else:
        peel_score = 0

    # ---------- NEW: direct exposure by labels (value share) ----------
    # label_maps example:
    # {'sanctioned': set([...]), 'mixers': set([...]), 'bridges': set([...]), 'cex': set([...]), 'defi': set([...])}
    def _exposure(mask, window_mask):
        if not np.any(window_mask):
            return 0.0
        # value-weighted share: use absolute value (both in/out)
        wvals = np.abs(vals_eth[window_mask])
        total = wvals.sum()
        if total <= 0:
            return 0.0
        risky = wvals[window_mask & mask].sum()
        return float(risky / total)

    exposure = {
        "exposure_mixer_30d": 0.0, "exposure_mixer_90d": 0.0,
        "exposure_bridge_30d": 0.0, "exposure_bridge_90d": 0.0,
        "exposure_sanctioned_30d": 0.0, "exposure_sanctioned_90d": 0.0,
        "exposure_cex_30d": 0.0, "exposure_cex_90d": 0.0,
        "exposure_defi_30d": 0.0, "exposure_defi_90d": 0.0,
    }
    if label_maps:
        # mark a tx "touches set" if either from or to is in the set
        for key in ["mixers", "bridges", "sanctioned", "cex", "defi"]:
            s = set([a.lower() for a in label_maps.get(key, set())])
            mask_any = np.array([(a in s) or (b in s) for a, b in zip(froms, tos)], dtype=bool)
            exposure[f"exposure_{key[:-1]}_30d"] = _exposure(mask_any, m30)
            exposure[f"exposure_{key[:-1]}_90d"] = _exposure(mask_any, m90)

    # ---------- pack output (keeps your original keys) ----------
    base = {
        "total_transactions": total_txns,
        "self_transfer_ratio": float(self_transfers / total_txns) if total_txns else 0.0,
        "circular_txn_count": int(circular_count),
        "circular_txn_ratio": float(circular_count / total_txns) if total_txns else 0.0,
        "avg_txn_value_eth": avg_val,
        "txn_spike_score": int(spike),
        "value_std_dev": val_std,
        "avg_gas_used": gas_used_avg,
        "avg_gas_price": gas_price_avg,
        "active_days": days_active,
        "wallet_age_days": wallet_age_days,
        "unique_counterparties": int(len(counterparties)),
        "failed_txn_ratio": failed_ratio,
        "eth_inflow_outflow_ratio": float(inflow_wei / (outflow_wei + 1.0)),
        "erc20_txn_count": erc20_cnt,
        "nft_txn_count": nft_cnt,
        "first_txn_time_of_day": first_hour,
        "erc20_token_diversity": int(len(erc20_contract_set)),
        "tx_direction_ratio": tx_dir_ratio,
        "contract_interaction_ratio": float(contract_interactions / total_txns) if total_txns else 0.0,
        "value_entropy": val_entropy,
        "tx_burst_count": int(spike),
        "average_txn_interval": avg_interval,
        "new_token_interaction_count": int(len(new_token_contracts)),
        "token_approval_count": appr_cnt,
        "sbt_poap_event_count": sbt_cnt,
        "approved_token_list": list(erc20_contract_set),
    }

    new_feats = {
        # behavior
        "weekend_txn_ratio": weekend_ratio,
        "hour_entropy": hour_ent,
        "value_gini": gini_val,
        "burstiness_hour": burst_h,
        "burstiness_day": burst_d,
        "reciprocity_ratio": reciprocity,
        "back_and_forth_7d": back_forth_7d,
        "peel_chain_score_24h": peel_score,
        # windows
        "tx_count_7d": int(m7.sum()),
        "tx_count_30d": int(m30.sum()),
        "tx_count_90d": int(m90.sum()),
        "inflow_eth_7d": float(inflow_7),
        "inflow_eth_30d": float(inflow_30),
        "inflow_eth_90d": float(inflow_90),
        "outflow_eth_7d": float(out_7),
        "outflow_eth_30d": float(out_30),
        "outflow_eth_90d": float(out_90),
        # direct exposure (value-weighted)
        **exposure
    }

    base.update(new_feats)
    return base


In [ ]:
# Assuming your dataframe is named `combined_trx` and contains the 'Transactions' column
feature_df = combined_trx.apply(
    lambda r: extract_transaction_features(
        r['Transactions'],
        my_address=r['Address'].lower(),
        label_maps=LABEL_MAPS
    ),
    axis=1, result_type='expand'
)

# Merge the extracted features back to your original dataset
combined_trx = pd.concat([combined_trx, feature_df], axis=1)


In [ ]:
combined_trx.drop(columns=['transactions', 'noOfTrx', 'balance'], inplace=True, errors='ignore')
combined_trx.rename(columns={'noOfTrx.1': 'noOfTrx'})

,Address,Name,isSafe,Tags,Transactions,noOfTrx,Balance,total_transactions,self_transfer_ratio,circular_txn_count,...,erc20_token_diversity,tx_direction_ratio,contract_interaction_ratio,value_entropy,tx_burst_count,average_txn_interval,new_token_interaction_count,token_approval_count,sbt_poap_event_count,approved_token_list
0,0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731,NaN,True,NaN,"[{'blockNumber': '3509677', 'blockHash': '0x98...",19,83021670189587,19,0.000000,0,...,0,1.000000,0.0000,2.944439,1,1.188564e+07,4,0,0,[]
1,0xf4618a00ed016475ae4033bfff78946fec712f3b,NaN,True,NaN,"[{'blockNumber': '3032980', 'blockHash': '0xfd...",139,89999706252309863072,139,0.000000,0,...,2,1.909091,0.0000,3.344533,1,1.832666e+06,21,0,0,[]
2,0x0c3de458b51a11da7d4616f42f66c861e3859d3e,MEV Bot: 0x0c3...D3E,True,mev-bot,"[{'blockNumber': '15171537', 'blockHash': '0xb...",10000,0,10000,0.000000,0,...,0,0.250000,0.0001,0.020141,3,4.321025e+02,2,0,0,[]
3,0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4,NaN,True,NaN,"[{'blockNumber': '13623538', 'timeStamp': '163...",3,2757182116120516,3,0.000000,0,...,0,0.500000,0.0000,1.098612,1,3.460434e+07,2,0,0,[]
4,0x5d234535f2283c8e48a05ad9b7994077bdc743ab,NaN,True,NaN,"[{'blockNumber': '4023169', 'timeStamp': '1500...",28,54715310931929644663,28,0.000000,4,...,2,2.125000,0.0000,2.508245,2,4.097377e+06,17,2,0,"[0x0cf0ee63788a0849fe5297f3407f701e122cc023, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33744,0x221dd436df380a957b0fff2359549fd0368c6594,NaN,True,NaN,"[{'blockNumber': '4249885', 'timeStamp': '1504...",242,34181892106133550041,242,0.004132,13,...,58,1.296296,0.0000,2.558582,2,9.647335e+05,70,1,0,[0x4bd70556ae3f8a6ec6c4080a0c327b24325438f3]
33745,0xD33441A44Da0c1DCEE596aBB7ebDCdBa77c7DDBd,NaN,False,NaN,"[{'blockNumber': '5215208', 'timeStamp': '1520...",58,11118367256799,58,0.000000,1,...,1,0.666667,0.0000,3.788504,2,2.571977e+06,14,0,0,[]
33746,0x8aeda5b17366ebb9b9d3a4b91a1ac432c385606e,NaN,True,NaN,"[{'blockNumber': '4171844', 'timeStamp': '1503...",54,22714410955797866645,54,0.018519,0,...,2,1.333333,0.0000,3.341965,1,4.073114e+06,12,1,0,[0xdd974d5c2e2928dea5f71b9825b8b646686bd200]
33747,0x625bebea868a3053636a6719d683fd7e793fdb79,NaN,False,NaN,[],0,0,0,0.000000,0,...,0,0.000000,0.0000,0.000000,0,0.000000e+00,0,0,0,0


In [ ]:
output_file = '../../data/combined_wallets_with_transactions_and_balances_2.csv'
combined_trx.to_csv(output_file, index=False)

In [ ]:
combined_trx.head(10)

,Address,Name,isSafe,Tags,Transactions,noOfTrx.1,Balance,total_transactions,self_transfer_ratio,circular_txn_count,...,erc20_token_diversity,tx_direction_ratio,contract_interaction_ratio,value_entropy,tx_burst_count,average_txn_interval,new_token_interaction_count,token_approval_count,sbt_poap_event_count,approved_token_list
0,0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731,NaN,True,NaN,"[{'blockNumber': '3509677', 'blockHash': '0x98...",19,83021670189587,19,0.0,0,...,0,1.000000,0.0000,2.944439,1,1.188564e+07,4,0,0,[]
1,0xf4618a00ed016475ae4033bfff78946fec712f3b,NaN,True,NaN,"[{'blockNumber': '3032980', 'blockHash': '0xfd...",139,89999706252309863072,139,0.0,0,...,2,1.909091,0.0000,3.344533,1,1.832666e+06,21,0,0,[]
2,0x0c3de458b51a11da7d4616f42f66c861e3859d3e,MEV Bot: 0x0c3...D3E,True,mev-bot,"[{'blockNumber': '15171537', 'blockHash': '0xb...",10000,0,10000,0.0,0,...,0,0.250000,0.0001,0.020141,3,4.321025e+02,2,0,0,[]
3,0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4,NaN,True,NaN,"[{'blockNumber': '13623538', 'timeStamp': '163...",3,2757182116120516,3,0.0,0,...,0,0.500000,0.0000,1.098612,1,3.460434e+07,2,0,0,[]
4,0x5d234535f2283c8e48a05ad9b7994077bdc743ab,NaN,True,NaN,"[{'blockNumber': '4023169', 'timeStamp': '1500...",28,54715310931929644663,28,0.0,4,...,2,2.125000,0.0000,2.508245,2,4.097377e+06,17,2,0,"[0x0cf0ee63788a0849fe5297f3407f701e122cc023, 0..."
5,0xde05dcb77aff940dd524c056e4219786114e37c4,NaN,True,NaN,"[{'blockNumber': '9513459', 'timeStamp': '1582...",11,987499424634980857,11,0.0,0,...,0,1.000000,0.0000,1.420572,1,1.548741e+07,3,0,0,[]
6,0x24b1d27b0f6b5a2aa052acf59817a8d9e7a8600a,Titanbuilder: 0x24b...00A,True,proposer-fee-recipient,"[{'blockNumber': '15641680', 'blockHash': '0x7...",48,6355220135686935247,48,0.0,0,...,0,0.076923,0.0000,3.871201,1,1.539510e+06,1,0,0,[]
7,0x50fb734f0448a122b102f9a211e52c2f14eb208a,NaN,True,take-action,"[{'blockNumber': '13596972', 'blockHash': '0x9...",10,68023387756216,10,0.0,0,...,0,1.200000,0.0000,2.163956,1,3.379247e+06,6,1,0,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2]
8,0x886fc23811240f94e32fa66c748277f8f1b4a805,NaN,True,NaN,"[{'blockNumber': '11628046', 'blockHash': '0x1...",1,36474460710000000000,1,0.0,0,...,0,0.500000,0.0000,0.000000,1,0.000000e+00,1,0,0,[]
9,0x93737345d0395976751329feff0fb84fb13d58ec,NaN,True,NaN,"[{'blockNumber': '13824691', 'blockHash': '0xf...",10,0,10,0.0,0,...,0,1.333333,0.0000,1.748067,1,3.159520e+06,4,0,0,[]


In [ ]:
combined_trx.tail(10)

,Address,Name,isSafe,Tags,Transactions,noOfTrx.1,Balance,total_transactions,self_transfer_ratio,circular_txn_count,...,erc20_token_diversity,tx_direction_ratio,contract_interaction_ratio,value_entropy,tx_burst_count,average_txn_interval,new_token_interaction_count,token_approval_count,sbt_poap_event_count,approved_token_list
33739,0xaffe4546f6bd8732aebc357ef84e516d35b9ea02,NaN,False,NaN,"[{'blockNumber': '16518412', 'timeStamp': '167...",4,337006343982000,4,0.000000,0,...,0,1.333333,0.0,1.386294,1,8.104716e+06,4,0,0,[]
33740,0xe483245b181a6b09a27ac451a77f22a6cf84f704,Fake_Phishing3614,False,Phish / Hack,"[{'blockNumber': '10112376', 'timeStamp': '159...",3,1056000000000000,3,0.000000,0,...,0,0.500000,0.0,1.098612,1,4.894280e+05,2,0,0,[]
33741,0x4d07457f5a2678748f6d4c61421b6d7e2b89f8bb,Fake_Phishing3265,False,Phish / Hack,"[{'blockNumber': '9160057', 'timeStamp': '1577...",130,99922578000000000,130,0.000000,6,...,5,2.032258,0.0,4.737625,2,6.594961e+04,63,0,0,[]
33742,0x15830adde1901bb57579a03978c374b759005422,NaN,True,NaN,"[{'blockNumber': '11744966', 'timeStamp': '161...",130,40576487721000,130,0.000000,2,...,6,3.700000,0.0,1.737201,2,8.674173e+05,37,24,0,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0..."
33743,0x5269d0ba4afa22bc8a4845a5ec4a59c48f7b38e1,NaN,True,airdrop-hunter,"[{'blockNumber': '13489256', 'timeStamp': '163...",4,2169452242986000,4,0.000000,0,...,0,1.000000,0.0,1.386294,1,1.582141e+06,3,0,0,[]
33744,0x221dd436df380a957b0fff2359549fd0368c6594,NaN,True,NaN,"[{'blockNumber': '4249885', 'timeStamp': '1504...",242,34181892106133550041,242,0.004132,13,...,58,1.296296,0.0,2.558582,2,9.647335e+05,70,1,0,[0x4bd70556ae3f8a6ec6c4080a0c327b24325438f3]
33745,0xD33441A44Da0c1DCEE596aBB7ebDCdBa77c7DDBd,NaN,False,NaN,"[{'blockNumber': '5215208', 'timeStamp': '1520...",58,11118367256799,58,0.000000,1,...,1,0.666667,0.0,3.788504,2,2.571977e+06,14,0,0,[]
33746,0x8aeda5b17366ebb9b9d3a4b91a1ac432c385606e,NaN,True,NaN,"[{'blockNumber': '4171844', 'timeStamp': '1503...",54,22714410955797866645,54,0.018519,0,...,2,1.333333,0.0,3.341965,1,4.073114e+06,12,1,0,[0xdd974d5c2e2928dea5f71b9825b8b646686bd200]
33747,0x625bebea868a3053636a6719d683fd7e793fdb79,NaN,False,NaN,[],0,0,0,0.000000,0,...,0,0.000000,0.0,0.000000,0,0.000000e+00,0,0,0,0
33748,0x9bb86ab2ebe3e6b9b580d557f7adcc4ff746ec8f,NaN,True,NaN,"[{'blockNumber': '8002453', 'timeStamp': '1561...",2916,32616380022379778,2916,0.000686,48,...,48,7.727273,0.0,1.589994,6,6.124886e+04,340,532,1,"[0x78a0a62fba6fb21a83fe8a3433d44c73a4017a6f, 0..."


In [ ]:
combined_trx.columns

Index(['Address', 'Name', 'isSafe', 'Tags', 'Transactions', 'noOfTrx.1',
       'Balance', 'total_transactions', 'self_transfer_ratio',
       'circular_txn_count', 'circular_txn_ratio', 'avg_txn_value_eth',
       'txn_spike_score', 'value_std_dev', 'avg_gas_used', 'avg_gas_price',
       'active_days', 'wallet_age_days', 'unique_counterparties',
       'failed_txn_ratio', 'eth_inflow_outflow_ratio', 'erc20_txn_count',
       'nft_txn_count', 'first_txn_time_of_day', 'erc20_token_diversity',
       'tx_direction_ratio', 'contract_interaction_ratio', 'value_entropy',
       'tx_burst_count', 'average_txn_interval', 'new_token_interaction_count',
       'token_approval_count', 'sbt_poap_event_count', 'approved_token_list'],
      dtype='object')

In [ ]:
# combined_trx_2 = pd.read_csv('combined_wallets_with_transactions_and_balances_2.csv')
combined_trx_2 = combined_trx.copy()

In [ ]:
# Clean the Dataset
combined_trx_2.fillna({
    'Name': 'NA',
    'Tags': 'NA',
    'balance': 0,
    'noOfTrx': 0,
    'total_transactions': 0,
    'wallet_age_days': 0,
    'erc20_txn_count': 0,
    'nft_txn_count': 0
}, inplace=True)

from sklearn.preprocessing import MinMaxScaler

features_to_normalize = [
    'total_transactions',
    'self_transfer_ratio',
    'circular_txn_count',
    'circular_txn_ratio',
    'avg_txn_value_eth',
    'txn_spike_score',
    'value_std_dev',
    'avg_gas_used',
    'avg_gas_price',
    'active_days',
    'wallet_age_days',
    'unique_counterparties',
    'failed_txn_ratio',
    'eth_inflow_outflow_ratio',
    'erc20_txn_count',
    'nft_txn_count',
    'first_txn_time_of_day',
    'erc20_token_diversity',
    'tx_direction_ratio',
    'contract_interaction_ratio',
    'value_entropy',
    'tx_burst_count',
    'average_txn_interval',
    'new_token_interaction_count',
    'token_approval_count',
    'sbt_poap_event_count'
]


scaler = MinMaxScaler()
df_scaled = combined_trx_2.copy()
df_scaled[features_to_normalize] = scaler.fit_transform(combined_trx_2[features_to_normalize])


In [ ]:
output_file = '../../data/combined_wallets_with_transactions_and_balances_3.csv'
df_scaled.to_csv(output_file, index=False)

In [ ]:
df_scaled.head(10)

,Address,Name,isSafe,Tags,Transactions,noOfTrx.1,Balance,total_transactions,self_transfer_ratio,circular_txn_count,...,erc20_token_diversity,tx_direction_ratio,contract_interaction_ratio,value_entropy,tx_burst_count,average_txn_interval,new_token_interaction_count,token_approval_count,sbt_poap_event_count,approved_token_list
0,0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731,NA,True,NA,"[{'blockNumber': '3509677', 'blockHash': '0x98...",19,83021670189587,0.0019,0.0,0.000000,...,0.000000,0.000212,0.0000,0.319688,0.000805,0.047155,0.0004,0.000000,0.0,[]
1,0xf4618a00ed016475ae4033bfff78946fec712f3b,NA,True,NA,"[{'blockNumber': '3032980', 'blockHash': '0xfd...",139,89999706252309863072,0.0139,0.0,0.000000,...,0.001381,0.000405,0.0000,0.363128,0.000805,0.007271,0.0021,0.000000,0.0,[]
2,0x0c3de458b51a11da7d4616f42f66c861e3859d3e,MEV Bot: 0x0c3...D3E,True,mev-bot,"[{'blockNumber': '15171537', 'blockHash': '0xb...",10000,0,1.0000,0.0,0.000000,...,0.000000,0.000053,0.0001,0.002187,0.002415,0.000002,0.0002,0.000000,0.0,[]
3,0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4,NA,True,NA,"[{'blockNumber': '13623538', 'timeStamp': '163...",3,2757182116120516,0.0003,0.0,0.000000,...,0.000000,0.000106,0.0000,0.119280,0.000805,0.137288,0.0002,0.000000,0.0,[]
4,0x5d234535f2283c8e48a05ad9b7994077bdc743ab,NA,True,NA,"[{'blockNumber': '4023169', 'timeStamp': '1500...",28,54715310931929644663,0.0028,0.0,0.000404,...,0.001381,0.000451,0.0000,0.272329,0.001610,0.016256,0.0017,0.000236,0.0,"[0x0cf0ee63788a0849fe5297f3407f701e122cc023, 0..."
5,0xde05dcb77aff940dd524c056e4219786114e37c4,NA,True,NA,"[{'blockNumber': '9513459', 'timeStamp': '1582...",11,987499424634980857,0.0011,0.0,0.000000,...,0.000000,0.000212,0.0000,0.154237,0.000805,0.061444,0.0003,0.000000,0.0,[]
6,0x24b1d27b0f6b5a2aa052acf59817a8d9e7a8600a,Titanbuilder: 0x24b...00A,True,proposer-fee-recipient,"[{'blockNumber': '15641680', 'blockHash': '0x7...",48,6355220135686935247,0.0048,0.0,0.000000,...,0.000000,0.000016,0.0000,0.420310,0.000805,0.006108,0.0001,0.000000,0.0,[]
7,0x50fb734f0448a122b102f9a211e52c2f14eb208a,NA,True,take-action,"[{'blockNumber': '13596972', 'blockHash': '0x9...",10,68023387756216,0.0010,0.0,0.000000,...,0.000000,0.000255,0.0000,0.234949,0.000805,0.013407,0.0006,0.000118,0.0,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2]
8,0x886fc23811240f94e32fa66c748277f8f1b4a805,NA,True,NA,"[{'blockNumber': '11628046', 'blockHash': '0x1...",1,36474460710000000000,0.0001,0.0,0.000000,...,0.000000,0.000106,0.0000,0.000000,0.000805,0.000000,0.0001,0.000000,0.0,[]
9,0x93737345d0395976751329feff0fb84fb13d58ec,NA,True,NA,"[{'blockNumber': '13824691', 'blockHash': '0xf...",10,0,0.0010,0.0,0.000000,...,0.000000,0.000283,0.0000,0.189794,0.000805,0.012535,0.0004,0.000000,0.0,[]


In [ ]:
df_scaled.head(10)

,Address,Name,isSafe,Tags,Transactions,noOfTrx.1,Balance,total_transactions,self_transfer_ratio,circular_txn_count,...,erc20_token_diversity,tx_direction_ratio,contract_interaction_ratio,value_entropy,tx_burst_count,average_txn_interval,new_token_interaction_count,token_approval_count,sbt_poap_event_count,approved_token_list
0,0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731,NA,True,NA,"[{'blockNumber': '3509677', 'blockHash': '0x98...",19,83021670189587,0.0019,0.0,0.000000,...,0.000000,0.000212,0.0000,0.319688,0.000805,0.047155,0.0004,0.000000,0.0,[]
1,0xf4618a00ed016475ae4033bfff78946fec712f3b,NA,True,NA,"[{'blockNumber': '3032980', 'blockHash': '0xfd...",139,89999706252309863072,0.0139,0.0,0.000000,...,0.001381,0.000405,0.0000,0.363128,0.000805,0.007271,0.0021,0.000000,0.0,[]
2,0x0c3de458b51a11da7d4616f42f66c861e3859d3e,MEV Bot: 0x0c3...D3E,True,mev-bot,"[{'blockNumber': '15171537', 'blockHash': '0xb...",10000,0,1.0000,0.0,0.000000,...,0.000000,0.000053,0.0001,0.002187,0.002415,0.000002,0.0002,0.000000,0.0,[]
3,0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4,NA,True,NA,"[{'blockNumber': '13623538', 'timeStamp': '163...",3,2757182116120516,0.0003,0.0,0.000000,...,0.000000,0.000106,0.0000,0.119280,0.000805,0.137288,0.0002,0.000000,0.0,[]
4,0x5d234535f2283c8e48a05ad9b7994077bdc743ab,NA,True,NA,"[{'blockNumber': '4023169', 'timeStamp': '1500...",28,54715310931929644663,0.0028,0.0,0.000404,...,0.001381,0.000451,0.0000,0.272329,0.001610,0.016256,0.0017,0.000236,0.0,"[0x0cf0ee63788a0849fe5297f3407f701e122cc023, 0..."
5,0xde05dcb77aff940dd524c056e4219786114e37c4,NA,True,NA,"[{'blockNumber': '9513459', 'timeStamp': '1582...",11,987499424634980857,0.0011,0.0,0.000000,...,0.000000,0.000212,0.0000,0.154237,0.000805,0.061444,0.0003,0.000000,0.0,[]
6,0x24b1d27b0f6b5a2aa052acf59817a8d9e7a8600a,Titanbuilder: 0x24b...00A,True,proposer-fee-recipient,"[{'blockNumber': '15641680', 'blockHash': '0x7...",48,6355220135686935247,0.0048,0.0,0.000000,...,0.000000,0.000016,0.0000,0.420310,0.000805,0.006108,0.0001,0.000000,0.0,[]
7,0x50fb734f0448a122b102f9a211e52c2f14eb208a,NA,True,take-action,"[{'blockNumber': '13596972', 'blockHash': '0x9...",10,68023387756216,0.0010,0.0,0.000000,...,0.000000,0.000255,0.0000,0.234949,0.000805,0.013407,0.0006,0.000118,0.0,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2]
8,0x886fc23811240f94e32fa66c748277f8f1b4a805,NA,True,NA,"[{'blockNumber': '11628046', 'blockHash': '0x1...",1,36474460710000000000,0.0001,0.0,0.000000,...,0.000000,0.000106,0.0000,0.000000,0.000805,0.000000,0.0001,0.000000,0.0,[]
9,0x93737345d0395976751329feff0fb84fb13d58ec,NA,True,NA,"[{'blockNumber': '13824691', 'blockHash': '0xf...",10,0,0.0010,0.0,0.000000,...,0.000000,0.000283,0.0000,0.189794,0.000805,0.012535,0.0004,0.000000,0.0,[]


In [ ]:
df_scaled['isSafe'].value_counts()

isSafe
True     25933
False     7816
Name: count, dtype: int64

In [ ]:
df_scaled.columns

Index(['Address', 'Name', 'isSafe', 'Tags', 'Transactions', 'noOfTrx.1',
       'Balance', 'total_transactions', 'self_transfer_ratio',
       'circular_txn_count', 'circular_txn_ratio', 'avg_txn_value_eth',
       'txn_spike_score', 'value_std_dev', 'avg_gas_used', 'avg_gas_price',
       'active_days', 'wallet_age_days', 'unique_counterparties',
       'failed_txn_ratio', 'eth_inflow_outflow_ratio', 'erc20_txn_count',
       'nft_txn_count', 'first_txn_time_of_day', 'erc20_token_diversity',
       'tx_direction_ratio', 'contract_interaction_ratio', 'value_entropy',
       'tx_burst_count', 'average_txn_interval', 'new_token_interaction_count',
       'token_approval_count', 'sbt_poap_event_count', 'approved_token_list'],
      dtype='object')